In [ ]:
# default_exp simulators.chp

# CHP Simulator

> Implementation of the Aarenson/Gottesman CHP stabilizer simulator (ref: https://arxiv.org/pdf/quant-ph/0406196.pdf). This simulator updates the stabilizer generators which stabilize a n-qubit state initially in the $|00..0 \rangle$ state after each gate application. It implements the fundamental gates CNOT, Hadamard and Phase (S-gate) from which all other gates of the Clifford group can be generated. This simulator, most notably, cannot handle any continuous rotation or other gates outside the Clifford group thus it is only applicable for simulating stabilizer circuits. This implementation is based on the work of @Strilanc (ref: https://github.com/Strilanc/python-chp-stabilizer-simulator).

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from qsam.simulators.simulator import Simulator
import numpy as np
import random

ModuleNotFoundError: No module named 'qsam'

In [ ]:
#export

class CHP(Simulator):
    """A simple stabilizer simulator"""
    
    def __init__(self, n_qubits):
        self.n_qubits = n_qubits
        self._table = np.eye(2*n_qubits+1, dtype=bool)
        self._x = self._table[:, :self.n_qubits] # X stab. gens
        self._z = self._table[:, self.n_qubits:-1] # Z stab. gens
        self._r = self._table[:, -1] # glob. phase
        
    def Z(self, i):
        self.S(i)
        self.S(i)
        
    def X(self, i):
        self.H(i)
        self.Z(i)
        self.H(i)
        
    def Y(self, i):
        self.S(i)
        self.S(i)
        self.S(i)
        self.X(i)
        self.S(i)
        
    def CNOT(self, c, t):
        # XI -> XX, IX -> IX, XX -> XI
        # ZI -> ZI, IZ -> ZZ, ZZ -> IZ
        # YI -> YX, IY -> ZY, YY -> -ZX
        self._r ^= self._x[:, c] & self._z[:, t] & (self._x[:, t] ^ self._z[:, c] ^ True)
        self._x[:, t] ^= self._x[:, c] # X propagation
        self._z[:, c] ^= self._z[:, t] # Z propagation
        
    def H(self, i):
         # X -> Z, Z -> X, Y -> -Y
        self._r ^= self._x[:, i] & self._z[:, i]
        self._x[:, i] ^= self._z[:, i]
        self._z[:, i] ^= self._x[:, i]
        self._x[:, i] ^= self._z[:, i]
        
    def S(self, i):
        # X -> Y, Z -> Z, Y -> -X
        self._r ^= self._x[:, i] & self._z[:, i]
        self._z[:, i] ^= self._x[:, i]
        
    def measure(self, i):
        for q in range(self.n_qubits):
            if self._x[q + self.n_qubits, i]:
                return int(self._measure_random(i, q)) # X destabilizes qubit i
            else:
                return int(self._measure_determined(i)) # only Z destabilizes qubit i
            
    def _measure_random(self, i, q, bias=0.5):
        self._table[q, :] = self._table[q + self.n_qubits, :]
        self._table[q + self.n_qubits, :] = 0
        self._z[q + self.n_qubits, i] = 1
        self._r[q + self.n_qubits] = random.random() < bias
        
        for j in range(2 * self.n_qubits):
            if self._x[j, i] and j != q and j != q + self.n_qubits:
                self._row_mult(i, q)
                
        return self._r[q + self.n_qubits]
    
    def _measure_determined(self, i):
        self._table[-1, :] = 0
        for j in range(self.n_qubits):
            if self._x[j, i]:
                self._row_mult(-1, j + self.n_qubits)
        return self._r[-1]
    
    def _row_product_sign(self, i, k):
        pauli_phases = sum( pauli_product_phase(self._x[i, j], self._z[i, j], self._x[k, j], self._z[k,j]) for j in range(self.n_qubits) )
        p = (pauli_phases >> 1) & 1
        return bool(self._r[i] ^ self._r[k] ^ p)
    
    def _row_mult(self, i, k):
        self._r[i] = self._row_product_sign(i, k)
        self._x[i, :self.n_qubits] ^= self._x[k, :self.n_qubits]
        self._z[i, :self.n_qubits] ^= self._z[k, :self.n_qubits]
        
    def __str__(self):
        pass
    
def pauli_product_phase(x1, z1, x2, z2):
    if x1 and z1: return int(z2) - int(x2)
    elif x1: return z2 and 2 * int(x2) - 1
    elif z1: return x2 and 1 - 2 * int(z2)
    else: return 0

**TODO** Tests and explanation of implementation, implement printing of state.

0